# Preprocessing Datasets for Machine Learning 

## Introduction

In practice, the data acquired for real world problems are often incomplete, noisy, and inconsistent. A few percentage of non-clean data points may effect the final performance by a few percentage drop. If a few steps of preprocessing were taken in the right direction, then better results would be easily achievable. A good data preprocessing is a necessary step for good machine learning performance and it is widely accepted that preprocessing takes the bulk of the overall machine learning effort.

In addition to data "cleaning", certain algorithms require data features properties in certain ways, such as **normalized** and **standardized** to make the method work better. For example clustering approaches by distance measures require data features to be normalized. The following procedures are common steps in preprocessing:

* Data formatting, cleaning
* Discretization, one-hot encoding
* Data integration and transformation
* Data reduction

## Data Formatting and Cleaning

Machine learning frameworks, such as pandas, scikit-learn, Weka, expect dataset files to be in certain formats to be able to process them. The Comma Separated Values CSV is one of the most common file formats. When examining datasets sometimes we see the files might contain artifacts:

* single quotes in double quotes, i.e. "Cote d'Azor" or reversed? e.g. 'Cote d'Azor'
* single quotes to differentiate between strings and values. i.e. '1' or 1
* use of semicolon instead of commas e.g. 1;50;red; in a row

In addition to the data formats artifacts, we might also see:

* duplicates of data lines (why is this undesired?)
* missing values (marked as '?' in Weka or 'NaN' in pandas for numerical variables)
* incorrect entries (e.g. clerical errors)

Note that framework programs such as Weka learners are mature and strong enough to work with these problems without us cleaning them with a preprocessing stage. However, if we do the preprocessing ourselves, then we always increase the **quality of the dataset** and this helps the next stages of machine learning pipeline.

---

## Worked Example

Consider the breast cancer dataset file located at the module page, named module03_breast_cancer.csv. Load it with pandas library and check for (1.) duplicates, (2.) missing values, (3.) incorrect entries. In the following cells, for each problem that the dataset has, a correction is provided once the situation is determined.

In [ ]:
# Standard libraries we always include
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)

# Locate and load the data file
df = pd.read_csv('../datasets/module03_breast_cancer.csv')
print(f'#rows={len(df)} #columns={len(df.columns)}')

# Print some info and plots to have a feeling about the dataset
print(df.dtypes)

In [ ]:
df.head()

In [ ]:
def plot_bc(_df):  # Make sure use a '_variable' name to avoid shadowing variables in other cells
    g = sns.FacetGrid(_df, col='deg-malig', hue='recurrence')
    g.map(plt.scatter, 'age', 'tumor-size', alpha=.7)
    g.add_legend()
    plt.show()
#
plot_bc(df)

**Observe:** In the second plot what is that data point at age 250?? ...Hmmm.

## Duplicates

Let's check duplicate values in our dataset.

In [ ]:
# Check for duplicates, this adds a new column to the dataset
df["is_duplicate"]= df.duplicated()

# Note that when using f-strings, the internal quote character must be different, such as 'is_duplicate' above
print(f"#total= {len(df)}")
print(f"#duplicated= {len(df[df['is_duplicate']==True])}")

In [ ]:
# Print rows which have True in column 'is_duplicate'
df[df['is_duplicate']==True]

In [ ]:
# Drop the duplicate rows using index - best way to drop in pandas
index_to_drop = df[df['is_duplicate']==True].index
df.drop(index_to_drop, inplace=True)

# Remove the duplicate marker column
df.drop(columns='is_duplicate', inplace=True)
print(f'#total= {len(df)}')

**Observe:** Total number of rows (data points) reduced to 293

## Missing values

Let's impute missing values. If we do not handle missing values, then most of the times the ML algorithms will handle them internally.

The safest and most common approach: Use mean (or equally acceptable median) for numerical values; and mode for nominal values to impute missing values.

Mean:  𝑥¯=1𝑁∑𝑖=1𝑁𝑥𝑖 
Median:  𝑥̃ =𝑥[|𝑥|/2]+𝑥[|𝑥|/2+1]2 
Mode:  𝑥̂ =𝚊𝚛𝚐𝚖𝚊𝚡𝑥𝑓(𝑥)

In [ ]:
# Do we have NaN in our dataset?
df.isnull().any()

In [ ]:
# We do have NaN - three numerical variables - check first cell, it says float 64
display(df[df['age'].isnull()])
display(df[df['tumor-size'].isnull()])
display(df[df['inv-nodes'].isnull()])

In [ ]:
# Mean values of columns
print(f"mean-age= {np.mean(df['age'])}")
print(f"mean-tumor-size= {np.mean(df['tumor-size'])}")
print(f"mean-inv-nodes= {np.mean(df['inv-nodes'])}")

# Impute
df['age'] = df['age'].fillna(df['age'].mean())
df['tumor-size'] = df['tumor-size'].fillna(df['tumor-size'].mean())
df['inv-nodes'] = df['inv-nodes'].fillna(df['inv-nodes'].mean())

# Check with the previous cell results
display(df.loc[[24,25,26,27,28]])


## Missing nominal values

Finding missing values in nominal variables is more tricky. First let's look at the nominal variables and then see what kind of unique values these nominal variables take. i.e. this is the level of the nominal variable drawn from a finite alphabet. Unless a numerical type (int64, float64, etc) df.dtype will correspond to an object which is a np.object class after read into from a CSV file.

In [ ]:
# What are the column types?
df.dtypes

In [ ]:
# Check unique levels and see any marker is used for a missing level
for col in df.columns:
    if df[col].dtype == np.object:
        print(col, df[col].unique())

The variables node-caps and breast-quad has '?' levels which need to be imputed with values to help the preprocessing.

In [ ]:
# Check the next feature
display(df['node-caps'].value_counts())
print('mode-node-caps', df['node-caps'].value_counts().index[0])

In [ ]:
# Check the next feature
display(df['breast-quad'].value_counts())
print('mode-breast-quad', df['breast-quad'].value_counts().index[0])

In [ ]:
# Replace '?' with mode - value/level with highest frequency in the feature
df['node-caps'] = df['node-caps'].replace({'?':'no'})
df['breast-quad'] = df['breast-quad'].replace({'?':'left_low'})

In [ ]:
# Again, check unique levels and see any marker is used or left out for a missing level
for col in df.columns:
    if df[col].dtype == np.object:
        print (col, df[col].unique())

## Incorrect entries

Remember the age value 250 from previous cells?

Finding out incorrect entries is more difficult than the previous steps as incorrect entries truly depend on the data column and **domain knowledge.** For this step we will look at the plots of numerical columns and figure out possible incorrect entries. Also Subject Matter Experts (SME) would help greatly in real-world projects about incorrect entries.

Note that incorrect entries may not be corrected easily and sometimes the best might be to drop that data point.

In [ ]:
# Let's use kernel density estimation to color the density
from scipy.stats import gaussian_kde

# We will reuse this plotting function later
def plot_bc_numericals(_df):
    fig, axs = plt.subplots(1, 4, figsize=(18, 2.5), sharey=True)
    y = df['recurrence'].astype('category').cat.codes.ravel()
    xy = np.vstack([_df['age'],y]); z = gaussian_kde(xy)(xy)
    axs[0].scatter(_df['age'], _df['recurrence'], c=z, s=50, edgecolor=None)
    axs[0].set_xlabel('age')
    xy = np.vstack([_df['tumor-size'],y]); z = gaussian_kde(xy)(xy)
    axs[1].scatter(_df['tumor-size'], _df['recurrence'], c=z, s=50, edgecolor=None)
    axs[1].set_xlabel('tumor-size')
    xy = np.vstack([_df['inv-nodes'],y]); z = gaussian_kde(xy)(xy)
    axs[2].scatter(_df['inv-nodes'], _df['recurrence'], c=z, s=50, edgecolor=None)
    axs[2].set_xlabel('inv-nodes')
    xy = np.vstack([_df['deg-malig'],y]); z = gaussian_kde(xy)(xy)
    axs[3].scatter(_df['deg-malig'], _df['recurrence'], c=z, s=50, edgecolor=None)
    axs[3].set_xlabel('deg-malig')
    fig.suptitle('Breast-cancer dataset numerical variables')
    plt.show()

plot_bc_numericals(df)

In [ ]:
# Remove that line with the incorrect age=250 and age=-5
display(df[df['age']==250])
index_to_drop = df[df['age']==250].index
df.drop(index_to_drop, inplace=True)
index_to_drop = df[df['age']==-5].index
df.drop(index_to_drop, inplace=True)

# Check results
print(f'#total= {len(df)}')
plot_bc_numericals(df)

In [ ]:
# Let's reset the indices to the dataframe after dropping a few rows
df = df.reset_index(drop=True)

## Cleaning complete

Compare the previous two cells to see the effect of removing the incorrect age entry.

At this point we are ready to apply some learners such as the Random Forest classifier.

# Discretization

Discretization is the process where a numerical variable is mapped to some levels by binning. This step is a big research/engineering area in machine learning. Recall that an example was provided in the past modules where the target (dependent) variable was discretized into three levels.

For our purposes, in this step, we will do the post-discretization, and apply one hot encoding to a nominal/discretized variable. Note that the variable might be a nominal variable naturally, such as the 'breast' variable which takes values from the alphabet {'left', 'right'}.

Now, we would like to continue preparing (preprocess) the dataset further to meet the requirements of the classifier that we would like to use - Random Forest classifier from scikit-learn library. This classifier works only on numerical data, thus we will convert the nominal variables into one hot encoded numerical variables, as explained in previous modules.

In [ ]:
# pandas get_dummies function is the one-hot-encoder
def encode_onehot(_df, f):
    _df2 = pd.get_dummies(_df[f], prefix='', prefix_sep='').max(level=0, axis=1).add_prefix(f+' - ')
    df3 = pd.concat([_df, _df2], axis=1)
    df3 = df3.drop([f], axis=1)
    return df3

# Print nominal variables
for f in list(df.columns.values):
    if df[f].dtype == np.object:
        print(f) 

Question: Will we one-hot-encode the variable 'recurrence'?

In [ ]:
# Display the original
display(df['menopause'][:10])

# Apply the onehot-encoding method
df_o = encode_onehot(df, 'menopause')

# Check the onehot-encoded version of this feature
cols = []
for f in list(df_o.columns.values):
    if 'menopause' in f:
        cols += [f]

In [ ]:
# Display the onehot-encoded        
display(df_o[cols][:10])

In [ ]:
# Apply the rest of the nominal features too
df_o = encode_onehot(df_o, 'node-caps')
df_o = encode_onehot(df_o, 'breast')
df_o = encode_onehot(df_o, 'breast-quad')
df_o = encode_onehot(df_o, 'irradiat')

In [ ]:
# Let's check how many features we have
print(f'before={len(df.columns)}, after={len(df_o.columns)}')

In [ ]:
df_o.head()

Next, let's classify the preprocessed dataset using the following strategies:

* 80% random test-train split
* Leave-one-out
* 10-fold cross validation
* Stratified 10-fold cross validation

Note that the target variable is binary, predicting when the cancer is recurred, or the cancer did not recur. Clearly this dataset has ground truth captured from the data source, or in other words, dataset is pre-labeled, or carry the ground truth. Thus we will employ **supervised learning**.

**Important:** Do not forget to remove the target (predicted, dependent) variable from X. Remember the Dataframe we are working already has the target variable and we will move it to y vector.

In [ ]:
# Show that the dependent variable is unbalanced
display(df['recurrence'].value_counts())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

# We will reuse the classifier function below
def rf_train_test(_X_tr, _X_ts, _y_tr, _y_ts):
    # Create a new random forest classifier, with working 4 parallel cores
    rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=None, n_jobs=4)
    # Train on training data
    model = rf.fit(_X_tr, _y_tr)
    # Test on training data
    y_pred = rf.predict(_X_ts)
    # Return accuracy
    return accuracy_score(_y_ts, y_pred)

In [ ]:
# Prepare the input X matrix and target y vector
X = df_o.loc[:, df_o.columns != 'recurrence'].values
y = df_o.loc[:, df_o.columns == 'recurrence'].values.ravel()

In [ ]:
# Sanity check
print(y[:10])

In [ ]:
# 80% split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)
rf_train_test(X_train, X_test, y_train, y_test)

**Question:** What will be the performance (accuracy) when we run the above cell again? Will you see any variations?

In [ ]:
# Run 10 times
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)
    print(rf_train_test(X_train, X_test, y_train, y_test))

**Important:** As the training and testing partition changes, the performance follows respectively.



**Question:** How can we measure the performance so that we can be sure of reporting it right?



In [ ]:
%%time
# Run 100 times and collect statistics
accuracies = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)
    accuracies += [rf_train_test(X_train, X_test, y_train, y_test)]
#
print(f'80% train-test split accuracy is {np.mean(accuracies):.3f} {chr(177)}{np.std(accuracies):.4f}')

**Note:** Leave-one-out evaluation keeps a single data point and label for test and uses all except for the test vector for training. Then, the evaluation process repeats this for each of the remaining data points, having a total number of  𝑁  accuracies.

The sklearn API says train and test require a 2D X and 1D y even when there is only one data point. Below code generates the test vectors properly.

In [ ]:
%%time
# Leave one out testing - this takes relatively longer
N = X.shape[0]
accuracies = []
for i in range (0,N):
    # Keep the 2D vector for the single test data point X
    X_test = X[i].reshape(1, -1)
    X_train = np.delete(np.array(X, copy=True), i, axis=0)
    # Keep the 1D vector for the single test label y
    y_test = [y[i]]
    y_train = np.delete(np.array(y, copy=True), i, axis=0)
    accuracies += [rf_train_test(X_train, X_test, y_train, y_test)]
#
# Sanity
print(f'Leave-one-out accuracy N= {N}, #accuracies= {len(accuracies)}')
# Score
print(f'Leave-one-out accuracy is {np.mean(accuracies):.3f} {chr(177)}{np.std(accuracies):.4f}')

In [ ]:
%%time
# 10-fold cross validation
accuracies = []
kf = KFold(n_splits=10,shuffle=False,random_state=None)
for train_index, test_index in kf.split(X, y):
    acc = rf_train_test(X[train_index], X[test_index], y[train_index], y[test_index])
    accuracies += [acc]
#
print(f'10-fold cross validation accuracy is {np.mean(accuracies):.3f} {chr(177)}{np.std(accuracies):.4f}')

In [ ]:
%%time
def eval_classifier(X, y, niter):
    accuracies = []
    kf = StratifiedKFold(n_splits=10,shuffle=False,random_state=None)
    for train_index, test_index in kf.split(X, y):
        acc = rf_train_test(X[train_index], X[test_index], y[train_index], y[test_index])
        accuracies += [acc]
    print( (f'Stratified 10-fold cross validation accuracy is '
            f'{np.mean(accuracies):.3f} {chr(177)}{np.std(accuracies):.4f} with {niter} total iterations')
         )
#
eval_classifier(X, y, 100)

Note the above performance results for discussion in the following cells.



In [ ]:
**Question:** What are the differences between these four evaluation methods?

In [ ]:
**Question:** What are the differences between these four evaluation methods

**Question:** What are the differences between these four evaluation methods

## Data Transformation

Now that we preprocessed and used the data for classification we can move to other interesting problems.

Imagine, we did not have the ground truth, so that a supervised learning was not possible. A natural approach in this case is clustering the data to see if there are some patterns or models we can come up with that explains the cancer behavior. We will attempt answering questions like "Is there a direct relation between menopause and cancer?"

First, let's draw some plots where the x, y and z-dimensions are 'age', 'tumor-size', 'inv-nodes' and color is 'recurrence'.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Deep copy original dataframe
df2 = df.copy()

# Convert every feature to numbers
df2['recurrence'] = df['recurrence'].astype("category").cat.codes

df2['menopause'] = df['menopause'].astype("category").cat.codes.astype('float')
df2['node-caps'] = df['node-caps'].astype("category").cat.codes.astype('float')
df2['breast'] = df['breast'].astype("category").cat.codes.astype('float')
df2['breast-quad'] = df['breast-quad'].astype("category").cat.codes.astype('float')
df2['irradiat'] = df['irradiat'].astype("category").cat.codes.astype('float')

df2['deg-malig'] = df['deg-malig'].astype('float')

def draw3d(df, _mn, _mx):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim3d(_mn, _mx)
    ax.set_ylim3d(_mn, _mx)
    ax.set_zlim3d(_mn, _mx)
    ax.set_ylim(ax.get_ylim()[::-1]) 
    ax.scatter(df['age'], df['tumor-size'], df['inv-nodes'], c=df['recurrence'], s=30)
    ax.set_xlabel('age'); ax.set_ylabel('tumor-size'); ax.set_zlabel('inv-nodes')
#
draw3d(df2, 0, 100)

Question: Do the dimensions 'age', 'tumor-size', 'inv-nodes' look fine in the above 3D plot?

Answer: The features are clumped and not nicely occupy  [0−100]  range, i.e. we are not seeing a spherical cluster shape.

---

Let's cluster the cancer data, without using the ground truth. We have to convert the nominal variables to numerical by using the category codes like we applied to 'recurrence' variable.

Important: Make sure every variable is of the same type, e.g. float32.

Important: Note that the values 'recurrence' took {0,1}, and by looking at the 3d plot above, can we easily find out which values (0 or 1) corresponds to 'recurrence-events' levels?

In [ ]:
from sklearn.cluster import KMeans

def kmeans(_X, _y, niter):  # do it niter times to collect statistics
    accuracies = []
    for i in range(niter):
        # We know that there are two levels in tagret variable - thus n_clusters=2
        km = KMeans(n_clusters=2, random_state=None)
        clusters = km.fit_predict(_X)
        accuracies += [accuracy_score(_y, clusters)]
    #
    return np.mean(accuracies)

X = df2.loc[:, df2.columns != 'recurrence'].values
y = df2.loc[:, df2.columns == 'recurrence'].values.ravel()

print(f'Clustering error= {kmeans(X, y, 100):.3f}')

Above performance is not very good as the error is almost equivalent to random choice, which would be  12  since we have 2 classes.

## Normalization and Standardization

Mapping the values of a column to  [0,1]  range is normalization:  𝑥𝑖−min(𝑥)max(𝑥)−min(𝑥)

Standardization is mapping the values to a  0 -mean  1 -standard-deviation distribution:  𝑥𝑖−mean(𝑥)stdev(𝑥)

Normalization makes the optimization surface more spherical, which helps the optimizer using each feature with equal importance. This is especially important and helping for distance based approaches.

Let's try two scalers from sklearn.preprocessing 1.Normalization MinMaxScaler(), 2. Standardization scale()

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
df2[['age', 'tumor-size', 'inv-nodes']] = min_max_scaler.fit_transform(df2[['age', 'tumor-size', 'inv-nodes']])

draw3d(df2, 0, 1)

By normalizing the values through expansion and contraction to  [0,1]  we achieve the distance between the data points are in the same "range" or unit. Thus the distance metrics like Euclidean distance will weigh each dimension or feature equally.

Example: Imagine a dataset which has speed in miles  [0,100]  and time traveled in seconds  [0,43200]  (12 hours max). A proper approach would be mapping both features into  [0,1]  scale to treat the feature space spherically. For actual feature values an inverse transformation can be used to map back to the original units (for example to be presented to user).

A distance metric  𝑑  in  𝑀  dimensions (Dataframe has M number of columns) such as Euclidean  𝑑𝑖𝑘=∑𝑀𝑗=0(𝑥𝑖𝑗−𝑥𝑘𝑗)2‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾√

As an example, clustering algorithms use some form of distance metric such as Euclidean distance between pairs of data points.

As can be seen from above example, normalization of variables is a necessary step for clustering.

In [ ]:
df2[['deg-malig', 'breast-quad']] = min_max_scaler.fit_transform(df2[['deg-malig', 'breast-quad']])

X = df2.loc[:, df2.columns != 'recurrence'].values
y = df2.loc[:, df2.columns == 'recurrence'].values.ravel()

print(f'Clustering error= {kmeans(X, y, 100):.3f}')

And now standardization.



In [ ]:
df2[['age', 'tumor-size', 'inv-nodes']] = preprocessing.scale(df2[['age', 'tumor-size', 'inv-nodes']])

draw3d(df2, 0, 10)

In [ ]:
df2.head()


In [ ]:
df2[['deg-malig', 'breast-quad']] = preprocessing.scale(df2[['deg-malig', 'breast-quad']])

X = df2.loc[:, df2.columns != 'recurrence'].values
y = df2.loc[:, df2.columns == 'recurrence'].values.ravel()

print(f'Clustering error= {kmeans(X, y, 100):.3f}')

In [ ]:
# Scaled
plot_bc(df2)

**Question:** Do you see any difference/improvement on the variables compared to the first set of plots in cell 1, repeated below?

**Answer:** Shapes are same but axis scales are different.

In [ ]:
# Original
plot_bc(df)

Note that after variable transformation, variables become more spherical or Gaussian like, but then the levels or data points do not correspond to any meaningful value in the domain knowledge that the dataset originally belonged to. For example 'deg-malig' had three levels {1, 2, 3} which probably meant something to the doctors dealing with cancer patients. However depending on the dataset, such transformations make a difference, albeit a few percentage improvement on the performance.

## Data Reduction
Reducing the data helps in a few ways:

* Faster method run-time, such as training
* More generalized models, decreases overfitting
* Simpler models that make more sense to the domain expert or Subject Matter Expert (SME)
* In some cases better accuracy performance - not necessarily always happens

**Feature ranking** and **feature selection** is a common stage that is executed after cleaning and preprocessing the data. In the following cells we will examine the variable rankings by **Univariate Feature Selection**.

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif


selector = SelectPercentile(f_classif, percentile=10)
# Fit the data
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()

# Display
cols = list(df2.loc[:, df2.columns != 'recurrence'].columns.values)
y_pos = np.arange(len(cols))
plt.bar(y_pos, scores)
plt.xticks(y_pos, cols, rotation=90)
plt.show()

Question: Can we drop 'age', 'menopause', 'breast', 'breast-quad' variables and redo the classification evaluation without a performance loss?

In [ ]:
df3 = df2.copy()
df3.drop(columns='age', inplace=True)
df3.drop(columns='menopause', inplace=True)
df3.drop(columns='breast', inplace=True)
df3.drop(columns='breast-quad', inplace=True)

X = df3.loc[:, df3.columns != 'recurrence'].values
y = df3.loc[:, df3.columns == 'recurrence'].values.ravel()

In [ ]:
eval_classifier(X, y, 100)


Wow! The performance accuracy did not drop. And we have less data columns now.



Note that we had standardized the data in the previous steps. Let's go back to the original dataset just after the cleaning was completed.

In [ ]:
df4 = df_o.copy()
df4.drop(columns='age', inplace=True)

# 'menopause' was onehot-encoded
for col in df4.columns.values:
    if 'menopause' in col:
        df4.drop(columns=col, inplace=True)

# 'breast' was onehot-encoded
for col in df4.columns.values:
    if 'breast' in col:
        df4.drop(columns=col, inplace=True)

# 'breast-quad' was onehot-encoded
for col in df4.columns.values:
    if 'breast-quad' in col:
        df4.drop(columns=col, inplace=True)

X = df4.loc[:, df4.columns != 'recurrence'].values
y = df4.loc[:, df4.columns == 'recurrence'].values.ravel()

In [ ]:
eval_classifier(X, y, 100)


More success! The performance accuracy increased! Or did we bias it?



Harder Question: Do you accept the performance increase as a valid increase? Or would you attribute it to the variance of error?



---

Question: What is the most important take-away in this effort?

